In [ ]:
!pip install pytorch-lightning==1.6.4

In [10]:
from pytorch_lightning import LightningModule
from torch.nn import Conv2d, Tanh, Sequential, AvgPool3d, Linear

In [11]:
in_channels = 65  # third dimension number 
output_features = 5  # number of chroma features 


class LeNet(LightningModule):
    def __init__():
        super(LeNet, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.regressor = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=output_features),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.regressor(x)
    
    def _common_step(self, batch, batch_idx, btype):
        not_training = btype != 'train'
        inpt, target = batch
        pred = self(inpt)
        loss = MSELoss()(pred, target)
        self.log(f'{btype}/{btype}_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True, reduce_fx=torch.mean, sync_dist=not_training)
        return loss
    
    def training_step(self, batch, batch_idx):
        loss = self._common_step(batch, batch_idx, 'train')
        return loss

    def validation_step(self, batch, batch_idx):
        self._common_step(batch, batch_idx, 'val')

    def test_step(self, batch, batch_idx):
        self._common_step(batch, batch_idx, 'test')

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), eps=1e-07, lr=1e-3, weight_decay=0)
        return optimizer
